# 팀 코드

본 노트북은 지금까지 했던 과정을 간략히 정리한 것입니다. 참고용으로 보시기 바랍니다.

# 제출 기록

1. D-6
 - 파생 변수 생성 확인
    1. errtype을 var로 생성 : 0.83068(기각)
    2. errtype, errcode 유니크 개수: 0.83235(기각)
    3. errtype 유니크 개수: 0.83261(기각)
    

2. D-5
 - 파생 변수 생성 확인
    1. err, qual의 time을 second으로 하고 std 로 groupby : 0.83490(상승)
    2. 혜인이가 추천한 중요도 높은 errcode 추가: 0.83447(기각)
    3. quality-groupby-sum: 0.83518(상승)

3. D-4
 - 베스트 모델 확인
    1. catb: 0.83552(상승)(2위)
    2. gbc: 0.82778(기각)(3위)
    3. ~~lgbm-1: 0.83239(기각)~~

4. D-3
 - 베스트 모델 확인
    1. lgbm-2: 0.83569(상승)(1위)
    2. xtree: 0.81803(기각)(6위)
    3. rf: 0.82260(기각)(5위)

5. D-2
 - 앙상블 제출
    1. ngb: 0.82469(기각)(4위)
    2. 최고모델에 혜인이 변수 조정 버전: 	0.83746(상승)
    3. best-6: 	0.83395(기각)

6. D-1
 - 앙상블 제출
    1. best-5(xtree 제외): 
    2. best-4(xtree, rf 제외):
    3. best-3(xtree, rf, ngb 제외):

7. D-Day
 - 보류

In [1]:
import load_dtypes as ld
import warnings
warnings.filterwarnings(action='ignore')

import os, sys
import time
import datetime as dt

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm

from imblearn.over_sampling import SMOTE, BorderlineSMOTE, ADASYN

from sklearn.model_selection import train_test_split
from sklearn.metrics import auc, roc_auc_score, roc_curve, precision_recall_curve, recall_score, precision_score

from ngboost import NGBClassifier
from ngboost.distns import k_categorical, Bernoulli
from ngboost.scores import LogScore, CRPScore
import lightgbm as lgb
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier

from sklearn.model_selection import StratifiedKFold

In [2]:
TRAIN_P_PATH = r'C:\Users\Wyatt\wyatt37/Data/systemError/train_problem_data.csv'
TRAIN_Q_PATH = r'C:\Users\Wyatt\wyatt37/Data/systemError/train_quality_data.csv'
TRAIN_E_PATH = r'C:\Users\Wyatt\wyatt37/Data/systemError/train_err_data.csv'
TEST_Q_PATH = r'C:\Users\Wyatt\wyatt37/Data/systemError/test_quality_data.csv'
TEST_E_PATH = r'C:\Users\Wyatt\wyatt37/Data/systemError/test_err_data.csv'
SUBMISSION_PATH = r'C:\Users\Wyatt\wyatt37/Data/systemError/sample_submission.csv'

In [3]:
%%time
train_p = ld.load_dtypes(TRAIN_P_PATH)
train_q = ld.load_dtypes(TRAIN_Q_PATH)
train_e = ld.load_dtypes(TRAIN_E_PATH)
test_q = ld.load_dtypes(TEST_Q_PATH)
test_e = ld.load_dtypes(TEST_E_PATH)
submission = pd.read_csv(SUBMISSION_PATH)

C:\Users\Wyatt\wyatt37/Data/systemError/train_problem_data.csv
C:\Users\Wyatt\wyatt37/Data/systemError/train_quality_data.csv
C:\Users\Wyatt\wyatt37/Data/systemError/train_err_data.csv
C:\Users\Wyatt\wyatt37/Data/systemError/test_quality_data.csv
C:\Users\Wyatt\wyatt37/Data/systemError/test_err_data.csv
Wall time: 1min 16s


In [4]:
train_p.shape, train_q.shape, train_e.shape, test_q.shape, test_e.shape, submission.shape

((5429, 2),
 (828624, 16),
 (16554663, 6),
 (747972, 16),
 (16532648, 6),
 (14999, 2))

# Preprocessing

In [5]:
def preprocessing_problem(df, object_='binary'):
    """
    definition:
    train_problem 테이블을 받아서 target 값으로 변환
    1. {0, 1}의 binary로 변환
    2. {0 ~ n}의 multiclass로 변환
    """
    
    # 10001부터 24999까지의 index를 만들어줍니다.
    user_id_idx = np.array(range(10000, 25000, 1))
    
    # train_new_p라는 새로운 df를 만들고 index는 위에서 만든 user_id_idx 로 지정해줍니다.
    new_p = pd.DataFrame(index = user_id_idx)
    new_p['target'] = 0
    
    if object_ == 'binary':
        new_p.iloc[df.user_id.unique()-10000] = 1
        new_p = new_p.reset_index()
        new_p.rename({'index':'user_id'}, axis=1, inplace=True)
        
    elif object_ == 'multi':
        # multi는 count()로 집계를 해줍니다.
        new_p['target'] = df.groupby('user_id')['time'].count()
        new_p = new_p.fillna(0)
        new_p = new_p.reset_index()
        new_p.rename({'index':'user_id'}, axis=1, inplace=True)
        
    return new_p

In [6]:
train_b_p = preprocessing_problem(train_p, 'binary')
train_m_p = preprocessing_problem(train_p, 'multi')

In [7]:
def preprocessing_quality(df):
    """
    definition:
    EDA를 통해 알아낸 정보로 train_q, test_q를 정리해서 내뿜어줍니다.
    1. qaulity_3, quality_4 를 drop 합니다.(단일 value)
    2. qaulity_k 변수들을 정수로 encoding 합니다.
    """
    # 먼저 3, 4번을 drop 합니다.
    df.drop(['quality_3', 'quality_4'], axis=1, inplace=True)
    
    # qual 변수만 할당해주고, 정수로 형변환 해줍니다.
    columns = train_q.columns[train_q.columns.str.contains('quality')]
    # for문을 통해 각 column을 반복 작업해줍니다.
    for col in columns:
        try:
            if df[col].dtype == 'float32': # 기존에 float은 패스
                df[col] = df[col].fillna(-2)
            elif df[col].dtype == 'int8' or df[col].dtype == 'int16': # 기존에 int도 패스
                df[col] = df[col].fillna(-2)
                #print(col)
            else:
                df[col] = df[col].astype('object')
                # nan값이 있으면 float으로 갈 수 없으니 '-2' 으로 채워줍니다.
                df[col] = df[col].fillna('-2')
                df[col] = df[col].apply(lambda x: x.replace(',' , ''))
                df[col] = df[col].astype(np.float32)
        except:
            pass
        
    # fwver 에서 null 값이 꽤 있습니다. missing으로 채우겠습니다.
    df.fwver = df.fwver.astype('object')
    df.fwver = df.fwver.fillna('missing')
    df.fwver = df.fwver.astype('category')
        
    return df

In [8]:
train_q = preprocessing_quality(train_q)
test_q = preprocessing_quality(test_q)

In [9]:
def preprocessing_fwver(df):
    """
    definition:
    별건 아니고, e-set에 fwver 변수에서 '10' 이라는 값이 있는데, 이게 errtype이랑 겹쳐요.
    그래서 10을 -> 8.5.2 으로 바꿔줄 겁니다.
    굳이 이렇게 바꾸는 이유는, 해당 fw가 8.5.3버전과 같은 model_nm을 공유하기 때문입니다.
    """
    df.fwver = df.fwver.replace('10', '8.5.2')
        
    return df

In [10]:
train_e = preprocessing_fwver(train_e)
test_e = preprocessing_fwver(test_e)

In [11]:
def make_datetime(df):
    """
    definition:
    'time' column이 str로 되어 있으니, datetime으로 바꿔주는 함수입니다.
    다만 'time'양식이 pandas함수에 적용이 안되니, 강제로 슬라이싱해서 만들어줘야 합니다.
    
    """
    
    df['year'] = df['time'].apply(lambda x: str(x)[:4])
    df['month'] = df['time'].apply(lambda x: str(x)[4:6])
    df['day'] = df['time'].apply(lambda x: str(x)[6:8])
    df['hour'] = df['time'].apply(lambda x: str(x)[8:10])
    df['minute'] = '00' # minute을 넣어주지 않으면 datetime이 완성이 안되니, 00으로 넣어줍니다.
    
    df['time'] = pd.to_datetime(df.year + df.month + df.day + df.hour + df.minute)
    
    return df

In [12]:
%%time
train_p = make_datetime(train_p)
train_q = make_datetime(train_q)
test_q = make_datetime(test_q)
train_e = make_datetime(train_e)
test_e = make_datetime(test_e)

Wall time: 57.1 s


In [13]:
# groupby 연산에서 왜 문제가 생기나 했더니 category로 되어 있어서였습니다.

train_e['errtype'] = train_e.errtype.astype('object')
test_e['errtype'] = test_e.errtype.astype('object')

train_e['errcode'] = train_e.errcode.astype('object')
test_e['errcode'] = test_e.errcode.astype('object')

In [14]:
# fwver도 object로 잡아줍니다.

train_q.fwver = train_q.fwver.astype('object')
test_q.fwver = test_q.fwver.astype('object')

# Feature Engineering

## from Error_log

### 사용한 model의 개수

In [15]:
train_model_count = train_e[['user_id', 'model_nm']].drop_duplicates().groupby('user_id').count()
test_model_count = test_e[['user_id', 'model_nm']].drop_duplicates().groupby('user_id').count()

### 사용한 fwver 의 개수

In [16]:
train_fwver_count = train_e[['user_id', 'fwver']].drop_duplicates().groupby('user_id').count()
test_fwver_count = test_e[['user_id', 'fwver']].drop_duplicates().groupby('user_id').count()

### 경험한 error 의 총 개수

In [17]:
train_err_count = train_e.groupby('user_id')['errcode'].count()
test_err_count = test_e.groupby('user_id')['errcode'].count()

### 경험한 각 errtype의 value별 개수

In [18]:
# 33호
train_errcode_33 = train_e[train_e.errtype == 33][['user_id', 'errcode', 'hour']].groupby(['user_id', 'errcode']).count().unstack().fillna(0)
test_errcode_33 = test_e[test_e.errtype == 33][['user_id', 'errcode', 'hour']].groupby(['user_id', 'errcode']).count().unstack().fillna(0)

train_errcode_33.columns = ['err_33_1', 'err_33_2', 'err_33_3']
test_errcode_33.columns = ['err_33_1', 'err_33_2', 'err_33_3']

In [19]:
# 23호
train_errcode_23 = train_e[train_e.errtype == 23][['user_id', 'errcode', 'hour']].groupby(['user_id', 'errcode']).count().unstack().fillna(0)
test_errcode_23 = test_e[test_e.errtype == 23][['user_id', 'errcode', 'hour']].groupby(['user_id', 'errcode']).count().unstack().fillna(0)

train_errcode_23.columns = ['UNKNOWN', 'ACTIVE', 'connLMP', 'connESTA', 'connTO', 'connLOCAL', 'STANDBY', 'TERMINATE']
test_errcode_23.columns = ['UNKNOWN', 'ACTIVE', 'connLMP', 'connESTA', 'connTO', 'connLOCAL', 'STANDBY', 'TERMINATE']

In [20]:
# 34호
train_errcode_34 = train_e[train_e.errtype == 34][['user_id', 'errcode', 'hour']].groupby(['user_id', 'errcode']).count().unstack().fillna(0)
test_errcode_34 = test_e[test_e.errtype == 34][['user_id', 'errcode', 'hour']].groupby(['user_id', 'errcode']).count().unstack().fillna(0)

train_errcode_34.columns = ['err_34_1', 'err_34_2', 'err_34_3', 'err_34_4', 'err_34_5', 'err_34_6']
test_errcode_34.columns = ['err_34_1', 'err_34_2', 'err_34_3', 'err_34_4', 'err_34_5', 'err_34_6']

## from Quality_log

### 각 quality의 표준편차

In [21]:
train_qual_std = train_q.groupby(['user_id']).std()
test_qual_std = test_q.groupby(['user_id']).std()

new_columns = ['q_std_0', 'q_std_1', 'q_std_2', 'q_std_5', 'q_std_6', 'q_std_7', 'q_std_8', 'q_std_9', 'q_std_10','q_std_11', 'q_std_12']

train_qual_std.columns = new_columns
test_qual_std.columns = new_columns

### 기록한 quality log의 개수(12개당 1번)

In [22]:
train_qual_log = train_q.groupby('user_id')['time'].count()/12
test_qual_log = test_q.groupby('user_id')['time'].count()/12

### quality당 순수 개수

In [23]:
# 값을 만들기 위해 새로운 df를 받아옵니다.
train_q_temp = train_q.copy()
test_q_temp = test_q.copy()

# 0 값을 전부 nan 값으로 바꿔줍니다.
for i in [0, 1, 2, 5, 6, 7, 8, 9, 10, 11, 12]:
    train_q_temp.loc[train_q_temp['quality_{}'.format(i)] == 0, 'quality_{}'.format(i)] = np.nan
for i in [0, 1, 2, 5, 6, 7, 8, 9, 10, 11, 12]:
    test_q_temp.loc[test_q_temp['quality_{}'.format(i)] == 0, 'quality_{}'.format(i)] = np.nan

# 필요없는 변수들을 버려줍니다.
train_q_temp.drop(['time', 'fwver', 'year', 'month', 'day', 'hour', 'minute'], axis=1, inplace=True)
test_q_temp.drop(['time', 'fwver', 'year', 'month', 'day', 'hour', 'minute'], axis=1, inplace=True)

# 그룹바이 카운트 해줍니다.
train_qual_counts = train_q_temp.groupby('user_id').count()
test_qual_counts = test_q_temp.groupby('user_id').count()

# 겹치는 컬럼명을 바꿔줍니다.
train_qual_counts.columns = ['q_c_0', 'q_c_1', 'q_c_2', 'q_c_5', 'q_c_6',
                             'q_c_7', 'q_c_8', 'q_c_9', 'q_c_10', 'q_c_11', 'q_c_12']
test_qual_counts.columns = ['q_c_0', 'q_c_1', 'q_c_2', 'q_c_5', 'q_c_6',
                             'q_c_7', 'q_c_8', 'q_c_9', 'q_c_10', 'q_c_11', 'q_c_12']

### quality당 음수, 0에 대한 count

In [24]:
train_q_temp = train_q.copy()
test_q_temp = test_q.copy()

In [25]:
train_q_temp.drop(['time', 'fwver', 'year', 'month', 'day', 'hour', 'minute'], axis=1, inplace=True)
test_q_temp.drop(['time', 'fwver', 'year', 'month', 'day', 'hour', 'minute'], axis=1, inplace=True)

In [26]:
# 0 값만 count를 위해서 음수와 양수를 전부 nan으로 만들겠습니다.

for i in [0, 1, 2, 5, 6, 7, 8, 9, 10, 11, 12]:
    train_q_temp.loc[train_q_temp['quality_{}'.format(i)] < 0, 'quality_{}'.format(i)] = np.nan
    
for i in [0, 1, 2, 5, 6, 7, 8, 9, 10, 11, 12]:
    test_q_temp.loc[test_q_temp['quality_{}'.format(i)] < 0, 'quality_{}'.format(i)] = np.nan
    
for i in [0, 1, 2, 5, 6, 7, 8, 9, 10, 11, 12]:
    train_q_temp.loc[train_q_temp['quality_{}'.format(i)] > 0, 'quality_{}'.format(i)] = np.nan
    
for i in [0, 1, 2, 5, 6, 7, 8, 9, 10, 11, 12]:
    test_q_temp.loc[test_q_temp['quality_{}'.format(i)] > 0, 'quality_{}'.format(i)] = np.nan

In [27]:
train_qual_zeroCount = train_q_temp.groupby('user_id').count()
test_qual_zeroCount = test_q_temp.groupby('user_id').count()

In [28]:
new_columns = ['q_z_c_0', 'q_z_c_1', 'q_z_c_2', 'q_z_c_5', 'q_z_c_6', 'q_z_c_7', 'q_z_c_8', 'q_z_c_9', 'q_z_c_10','q_z_c_11', 'q_z_c_12']

train_qual_zeroCount.columns = new_columns
test_qual_zeroCount.columns = new_columns

In [29]:
train_q_temp = train_q.copy()
test_q_temp = test_q.copy()

In [30]:
train_q_temp.drop(['time', 'fwver', 'year', 'month', 'day', 'hour', 'minute'], axis=1, inplace=True)
test_q_temp.drop(['time', 'fwver', 'year', 'month', 'day', 'hour', 'minute'], axis=1, inplace=True)

In [31]:
# 음수 값만 count를 위해서 음수와 0을 전부 nan으로 만들겠습니다.

for i in [0, 1, 2, 5, 6, 7, 8, 9, 10, 11, 12]:
    train_q_temp.loc[train_q_temp['quality_{}'.format(i)] >= 0, 'quality_{}'.format(i)] = np.nan
    
for i in [0, 1, 2, 5, 6, 7, 8, 9, 10, 11, 12]:
    test_q_temp.loc[test_q_temp['quality_{}'.format(i)] >= 0, 'quality_{}'.format(i)] = np.nan

In [32]:
train_qual_negaCount = train_q_temp.groupby('user_id').count()
test_qual_negaCount = test_q_temp.groupby('user_id').count()

In [33]:
new_columns = ['q_n_c_0', 'q_n_c_1', 'q_n_c_2', 'q_n_c_5', 'q_n_c_6', 'q_n_c_7', 'q_n_c_8', 'q_n_c_9', 'q_n_c_10','q_n_c_11', 'q_n_c_12']

train_qual_negaCount.columns = new_columns
test_qual_negaCount.columns = new_columns

## Adding

### time에 대한 유저별 표준편차

In [34]:
import time, datetime

def time_to_seconds(x):
    return time.mktime(x.timetuple())

In [35]:
%%time
train_e.time = train_e.time.apply(lambda x: time_to_seconds(x))
test_e.time = test_e.time.apply(lambda x: time_to_seconds(x))
train_q.time = train_q.time.apply(lambda x: time_to_seconds(x))
test_q.time = test_q.time.apply(lambda x: time_to_seconds(x))

Wall time: 2min 9s


In [36]:
%%time
train_err_timestd = train_e.groupby(['user_id'])['time'].std()
test_err_timestd = test_e.groupby(['user_id'])['time'].std()
train_err_timestd = train_err_timestd.rename(level = 0, index = 't_e_std') 
test_err_timestd = test_err_timestd.rename(level = 0, index = 't_e_std') 

Wall time: 528 ms


In [37]:
%%time
train_qual_timestd = (train_q[['user_id', 'time']].drop_duplicates()).groupby(['user_id']).std()
test_qual_timestd = (test_q[['user_id', 'time']].drop_duplicates()).groupby(['user_id']).std()
train_qual_timestd.columns = ['t_q_std']
test_qual_timestd.columns = ['t_q_std']

Wall time: 95 ms


### quality를 sum으로 groupby

In [38]:
train_each_quality_sum = train_q.groupby('user_id').sum().loc[:, 'quality_0':]
test_each_quality_sum = test_q.groupby('user_id').sum().loc[:, 'quality_0':]

quality_sum_colnms = ['quality_0_sum', 'quality_1_sum', 'quality_2_sum', 'quality_5_sum', 'quality_6_sum', 
                      'quality_7_sum', 'quality_8_sum', 'quality_9_sum','quality_10_sum', 'quality_11_sum', 
                      'quality_12_sum']

train_each_quality_sum.columns = quality_sum_colnms
test_each_quality_sum.columns = quality_sum_colnms

# Train & Predict

### Base set

In [39]:
# errtype을 유저별로 카운트 해줍니다.

X = train_e.groupby(['user_id', 'errtype'])['errcode'].count().unstack().fillna(0)
y = test_e.groupby(['user_id', 'errtype'])['errcode'].count().unstack().fillna(0)

X.columns = ['et_1', 'et_2', 'et_3', 'et_4', 'et_5', 'et_6', 'et_7', 'et_8', 'et_9', 'et_10', 'et_11', 'et_12', 'et_13',
 'et_14', 'et_15', 'et_16', 'et_17', 'et_18', 'et_19', 'et_20', 'et_21', 'et_22', 'et_23', 'et_24', 'et_25',
 'et_26', 'et_27', 'et_28', 'et_30', 'et_31', 'et_32', 'et_33', 'et_34', 'et_35', 'et_36', 'et_37', 'et_38',
 'et_39', 'et_40', 'et_41', 'et_42']
y.columns = ['et_1', 'et_2', 'et_3', 'et_4', 'et_5', 'et_6', 'et_7', 'et_8', 'et_9', 'et_10', 'et_11', 'et_12', 'et_13',
 'et_14', 'et_15', 'et_16', 'et_17', 'et_18', 'et_19', 'et_20', 'et_21', 'et_22', 'et_23', 'et_24', 'et_25',
 'et_26', 'et_27', 'et_28', 'et_30', 'et_31', 'et_32', 'et_33', 'et_34', 'et_35', 'et_36', 'et_37', 'et_38',
 'et_39', 'et_40', 'et_41', 'et_42']

In [40]:
# 하나 사라진 유저를 채워주는 코드입니다.
y = y.reindex(pd.RangeIndex(y.index.max() + 1)).ffill(0)[30000:]

In [41]:
X.shape, y.shape

((15000, 41), (14999, 41))

### 1차 합병

In [42]:
X = pd.concat([X,
               train_err_count, # 유저가 기록한 총 err수
               train_fwver_count, # 유저가 사용한 fw수
               train_model_count, # 유저가 사용한 model 수
               train_qual_std, # 각 퀄리티에 대한 유저별 편차
               train_qual_log, # 유저별 퀄리티 로그의 수
               train_errcode_23, # 23호 에러타입의 밸류별 개수
               train_errcode_33, # 33호 상동
               train_errcode_34, # 34호 상동
               train_qual_counts, # 각 퀄리티에서 0을 제외한 카운트
               train_qual_negaCount, # 각 퀄리티에 대해 음수만 카운트
               train_qual_zeroCount # 각 퀄리티에 대해 0.만 카운트
              ], axis=1).fillna(0)
X.shape

(15000, 106)

In [43]:
y = pd.concat([y,
               test_err_count,
               test_fwver_count,
               test_model_count,
               test_qual_std,
               test_qual_log,
               test_errcode_23,
               test_errcode_33,
               test_errcode_34,
               test_qual_counts,
               test_qual_negaCount,
               test_qual_zeroCount
              ], axis=1).fillna(0)
y.shape

(14999, 106)

### 3차 합격생들

In [44]:
X = pd.concat([X, train_err_timestd, train_qual_timestd, train_each_quality_sum], axis=1).fillna(0)
y = pd.concat([y, test_err_timestd, test_qual_timestd, test_each_quality_sum], axis=1).fillna(0)

X.shape, y.shape

((15000, 119), (14999, 119))

### 3차 면접중

In [45]:
# 혜인이 다시 제안한 조건부 31호.
train_errcode_31 = train_e[train_e.errtype == 31][['user_id', 'errcode', 'time']].\
                            groupby(['user_id', 'errcode']).count().unstack().fillna(0)
test_errcode_31 = test_e[test_e.errtype == 31][['user_id', 'errcode', 'time']].\
                            groupby(['user_id', 'errcode']).count().unstack().fillna(0)
train_errcode_31.columns = ['err_31_0', 'err_31_1']
test_errcode_31.columns =['err_31_0', 'err_31_1']

In [46]:
%%time
train_e['day_2'] = train_e['time'].apply(lambda x: str(x)[:10])

train_meanDay = (train_e
                 .groupby(['user_id','day_2'])['day_2']
                 .count()
                 .unstack()
                 .fillna(0)
                 .loc[:, '2020-11-01':'2020-11-30']
                 .mean(axis=1))

train_maxDay = (train_e
                .groupby(['user_id','day_2'])['day_2']
                .count()
                .unstack()
                .fillna(0)
                .loc[:, '2020-11-01':'2020-11-30']
                .max(axis=1))

train_maxBymean = pd.Series(data = np.array(train_maxDay) / np.array(train_meanDay),
                            index = train_e.user_id.unique(),
                            name = 'mbm')

Wall time: 14.9 s


In [47]:
%%time
test_e['day_2'] = test_e['time'].apply(lambda x: str(x)[:10])

test_meanDay = (test_e
                .groupby(['user_id','day_2'])['day_2']
                .count()
                .unstack()
                .fillna(0)
                .loc[:, '2020-11-01':'2020-11-30']
                .mean(axis=1))

test_maxDay = (test_e
               .groupby(['user_id','day_2'])['day_2']
               .count()
               .unstack()
               .fillna(0)
               .loc[:, '2020-11-01':'2020-11-30']
               .max(axis=1))

test_maxBymean = pd.Series(data = np.array(test_maxDay) / np.array(test_meanDay),
                           index = test_e.user_id.unique(),
                           name = 'mbm')

Wall time: 15 s


In [48]:
X_temp = pd.concat([X, train_errcode_31, train_maxBymean], axis=1).fillna(0)
y_temp = pd.concat([y, test_errcode_31, test_maxBymean], axis=1).fillna(0)

X_temp.drop(['et_20', 'et_36'], axis=1, inplace = True)
y_temp.drop(['et_20', 'et_36'], axis=1, inplace = True)

X_temp.shape, y_temp.shape

((15000, 120), (14999, 120))

## Model

### Feature Selection: Ridge, Lasso

In [384]:
from sklearn.linear_model import RidgeCV, LassoCV, ElasticNetCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_auc_score

In [354]:
scaler = MinMaxScaler()

In [356]:
X_new_scaled = scaler.fit_transform(X_new)

In [379]:
a_list = [0.00001, 0.0001, 0.001, 0.01,
          0.05, 0.1, 0.2, 0.3, 0.5, 0.7, 0.9]

In [386]:
# Ridge
ridge = RidgeCV(alphas=a_list)
ridge.fit(X_new_scaled, train_b_p.target)

print('best alpha: ', ridge.alpha_)
print('feature selected: ', ridge.coef_.nonzero()[0].size)
print('R^2 is: ', ridge.score(X_new_scaled, train_b_p.target))
print('auc: ', roc_auc_score(train_b_p.target, ridge.predict(X_new_scaled)))

best alpha:  0.9
feature selected:  117
R^2 is:  0.21684725538690441
auc:  0.77278074


In [387]:
# Lasso
lasso = LassoCV(alphas=a_list)
lasso.fit(X_new_scaled, train_b_p.target)

print('best alpha: ', lasso.alpha_)
print('feature selected: ', lasso.coef_.nonzero()[0].size)
print('R^2 is: ', lasso.score(X_new_scaled, train_b_p.target))
print('auc: ', roc_auc_score(train_b_p.target, lasso.predict(X_new_scaled)))

best alpha:  0.0001
feature selected:  44
R^2 is:  0.21534357615160604
auc:  0.76866168


In [388]:
# elastic
elastic = ElasticNetCV(alphas=a_list)
elastic.fit(X_new_scaled, train_b_p.target)

print('best alpha: ', elastic.alpha_)
print('feature selected: ', elastic.coef_.nonzero()[0].size)
print('R^2 is: ', elastic.score(X_new_scaled, train_b_p.target))
print('auc: ', roc_auc_score(train_b_p.target, elastic.predict(X_new_scaled)))

best alpha:  0.0001
feature selected:  72
R^2 is:  0.2141987748183607
auc:  0.7707219


In [389]:
elastic.coef_.nonzero()[0]

array([  0,   4,   5,   6,   7,  10,  11,  12,  14,  15,  16,  17,  18,
        19,  20,  21,  22,  24,  25,  26,  27,  28,  30,  32,  33,  34,
        35,  37,  38,  39,  40,  41,  42,  43,  47,  48,  52,  53,  54,
        57,  61,  62,  63,  64,  65,  66,  67,  68,  70,  73,  74,  76,
        78,  79,  82,  84,  85,  86,  87,  88,  93,  95,  97,  98, 106,
       107, 111, 112, 113, 116, 117, 121], dtype=int64)

In [392]:
X_new.iloc[:, elastic.coef_.nonzero()[0]].columns

Index(['et_1', 'et_5', 'et_6', 'et_7', 'et_8', 'et_11', 'et_12', 'et_13',
       'et_15', 'et_16', 'et_17', 'et_18', 'et_19', 'et_20', 'et_21', 'et_22',
       'et_23', 'et_25', 'et_26', 'et_27', 'et_28', 'et_30', 'et_32', 'et_34',
       'et_35', 'et_36', 'et_37', 'et_39', 'et_40', 'et_41', 'et_42',
       'errcode', 'fwver', 'model_nm', 'q_std_5', 'q_std_6', 'q_std_10',
       'q_std_11', 'q_std_12', 'ACTIVE', 'connLOCAL', 'STANDBY', 'TERMINATE',
       'err_33_1', 'err_33_2', 'err_33_3', 'err_34_1', 'err_34_2', 'err_34_4',
       'q_c_0', 'q_c_1', 'q_c_5', 'q_c_7', 'q_c_8', 'q_c_11', 'q_n_c_0',
       'q_n_c_1', 'q_n_c_2', 'q_n_c_5', 'q_n_c_6', 'q_n_c_11', 'q_z_c_0',
       'q_z_c_2', 'q_z_c_5', 't_e_std', 't_q_std', 'quality_5_sum',
       'quality_6_sum', 'quality_7_sum', 'quality_10_sum', 'quality_11_sum',
       'user_max_by_mean'],
      dtype='object')

In [394]:
X_selected = X_new.iloc[:, elastic.coef_.nonzero()[0]]

### Base LGBM

In [395]:
#------------------------------------------------------------
# validation auc score를 확인하기 위해 정의
def f_pr_auc(probas_pred, y_true):
    labels=y_true.get_label()
    p, r, _ = precision_recall_curve(labels, probas_pred)
    score=auc(r,p) 
    return "pr_auc", score, True
#------------------------------------------------------------


def s_fold_train_pred(train_x, train_y):
    import lightgbm as lgb

    # Train
    models     = []
    recalls    = []
    precisions = []
    auc_scores   = []
    threshold = 0.5
    # 파라미터 설정
    params =      {
                    'boosting_type' : 'gbdt',
                    'objective'     : 'binary',
                    'metric'        : 'auc',
                    'learning_rate' : 0.027,
                    'seed': 42
                    }
    #-------------------------------------------------------------------------------------
    # 5 Kfold cross validation
    s_fold = StratifiedKFold(n_splits=5, shuffle=True ,random_state=42)    

    for train_idx, val_idx in s_fold.split(train_x, train_y):

        # split train, validation set
        X = train_x.iloc[train_idx]
        y = train_y.iloc[train_idx]
        valid_x = train_x.iloc[val_idx]
        valid_y = train_y.iloc[val_idx]

        d_train= lgb.Dataset(X, y)
        d_val  = lgb.Dataset(valid_x, valid_y)

        #run traning
        model = lgb.train(
                            params,
                            train_set       = d_train,
                            num_boost_round = 10000,
                            valid_sets      = d_val,
                            feval           = f_pr_auc,
                            verbose_eval    = 100, 
                            early_stopping_rounds = 100
                           )

        # cal valid prediction
        valid_prob = model.predict(valid_x)
        valid_pred = np.where(valid_prob > threshold, 1, 0)

        # cal scores
        recall    = recall_score(    valid_y, valid_pred)
        precision = precision_score( valid_y, valid_pred)
        auc_score = roc_auc_score(   valid_y, valid_prob)

        # append scores
        models.append(model)
        recalls.append(recall)
        precisions.append(precision)
        auc_scores.append(auc_score)

        print('==========================================================')
        
    return models, auc_scores, recalls, precisions

In [205]:
# loss 비교를 위해 지우지 않습니다.
# 최고점 모델입니다.
models, auc_scores, _, _ = s_fold_train_pred(X, train_b_p.target)
print(np.mean(auc_scores))

[LightGBM] [Info] Number of positive: 4000, number of negative: 8000
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003832 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 13551
[LightGBM] [Info] Number of data points in the train set: 12000, number of used features: 103
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.333333 -> initscore=-0.693147
[LightGBM] [Info] Start training from score -0.693147
Training until validation scores don't improve for 100 rounds
[100]	valid_0's auc: 0.826428	valid_0's pr_auc: 0.747213
[200]	valid_0's auc: 0.826504	valid_0's pr_auc: 0.74872
Early stopping, best iteration is:
[185]	valid_0's auc: 0.827018	valid_0's pr_auc: 0.748922
[LightGBM] [Info] Number of positive: 4000, number of negative: 8000
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004562 seconds.
You can set `force_col_wise=true` to remove the overhead.
[L

In [347]:
# 학습용
models, auc_scores, _, _ = s_fold_train_pred(X_new2, train_b_p.target)
print(np.mean(auc_scores))

[LightGBM] [Info] Number of positive: 4000, number of negative: 8000
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004421 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16104
[LightGBM] [Info] Number of data points in the train set: 12000, number of used features: 115
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.333333 -> initscore=-0.693147
[LightGBM] [Info] Start training from score -0.693147
Training until validation scores don't improve for 100 rounds
[100]	valid_0's auc: 0.826164	valid_0's pr_auc: 0.745748
[200]	valid_0's auc: 0.827937	valid_0's pr_auc: 0.750056
Early stopping, best iteration is:
[194]	valid_0's auc: 0.828017	valid_0's pr_auc: 0.750058
[LightGBM] [Info] Number of positive: 4000, number of negative: 8000
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004585 seconds.
You can set `force_col_wise=true` to remove the overhead.
[

In [396]:
# 학습용
models, auc_scores, _, _ = s_fold_train_pred(X_selected, train_b_p.target)
print(np.mean(auc_scores))

[LightGBM] [Info] Number of positive: 4000, number of negative: 8000
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003310 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 10214
[LightGBM] [Info] Number of data points in the train set: 12000, number of used features: 72
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.333333 -> initscore=-0.693147
[LightGBM] [Info] Start training from score -0.693147
Training until validation scores don't improve for 100 rounds
[100]	valid_0's auc: 0.821905	valid_0's pr_auc: 0.734777
[200]	valid_0's auc: 0.822917	valid_0's pr_auc: 0.741858
Early stopping, best iteration is:
[148]	valid_0's auc: 0.823461	valid_0's pr_auc: 0.74203
[LightGBM] [Info] Number of positive: 4000, number of negative: 8000
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002970 seconds.
You can set `force_col_wise=true` to remove the overhead.
[Li

In [201]:
# base 0.8292073  0.8266209  0.82655950
# ec   0.8296051  0.8269322  0.82608215
# et   0.8291249  0.8269684  0.82844335
# etc  0.8291123  0.8272698  0.82821499
#          42         43        44

In [240]:
pred_y_list = []
for model in models:
    pred_y = model.predict(y_temp)
    pred_y_list.append(pred_y.reshape(-1,1))
    
pred_ensemble = np.mean(pred_y_list, axis = 0)

In [241]:
submission.problem = pred_ensemble
submission

,user_id,problem
0,30000,0.920044
1,30001,0.221513
2,30002,0.505086
3,30003,0.735106
4,30004,0.880764
...,...,...
14994,44994,0.203552
14995,44995,0.320443
14996,44996,0.741580
14997,44997,0.884766


In [242]:
submission.to_csv(r'./submission/team_210129_3_qual-sum-added.csv', index=False)

### Tuned Catb

In [53]:
#------------------------------------------------------------
# validation auc score를 확인하기 위해 정의
def f_pr_auc(probas_pred, y_true):
    labels=y_true.get_label()
    p, r, _ = precision_recall_curve(labels, probas_pred)
    score=auc(r,p) 
    return "pr_auc", score, True
#------------------------------------------------------------


def catb_fold_train_pred(train_x, train_y):

    # Train
    models     = []
    recalls    = []
    precisions = []
    auc_scores   = []
    threshold = 0.5
    # 파라미터 설정
    params =      {
                    'nan_mode': 'Min',
                    'eval_metric': 'Logloss',
                    'iterations': 1000,
                    'sampling_frequency': 'PerTree',
                    'leaf_estimation_method': 'Newton',
                    'grow_policy': 'SymmetricTree',
                    'penalties_coefficient': 1,
                    'boosting_type': 'Plain',
                    'model_shrink_mode': 'Constant',
                    'feature_border_type': 'GreedyLogSum',
                    #'bayesian_matrix_reg': 0.10000000149011612,
                    'l2_leaf_reg': 3,
                    'random_strength': 1,
                    'rsm': 1,
                    'boost_from_average': False,
                    'model_size_reg': 0.5,
                    'subsample': 0.800000011920929,
                    'use_best_model': False,
                    'class_names': [0, 1],
                    'random_seed': 2584,
                    'depth': 6,
                    'posterior_sampling': False,
                    'border_count': 254,
                    'classes_count': 0,
                    'auto_class_weights': 'None',
                    'sparse_features_conflict_fraction': 0,
                    'leaf_estimation_backtracking': 'AnyImprovement',
                    'best_model_min_trees': 1,
                    'model_shrink_rate': 0,
                    'min_data_in_leaf': 1,
                    'loss_function': 'Logloss',
                    'learning_rate': 0.028116999194025993,
                    'score_function': 'Cosine',
                    'task_type': 'CPU',
                    'leaf_estimation_iterations': 10,
                    'bootstrap_type': 'MVS',
                    'max_leaves': 64
                    }
    #-------------------------------------------------------------------------------------
    # 5 Kfold cross validation
    s_fold = StratifiedKFold(n_splits=5, shuffle=True ,random_state=42)    

    for train_idx, val_idx in s_fold.split(train_x, train_y):

        # split train, validation set
        X = train_x.iloc[train_idx]
        y = train_y.iloc[train_idx]
        valid_x = train_x.iloc[val_idx]
        valid_y = train_y.iloc[val_idx]

        #run traning
        model = CatBoostClassifier(**params, verbose=0)
        model.fit(X, y)

        # cal valid prediction
        valid_prob = model.predict(valid_x)
        valid_pred = np.where(valid_prob > threshold, 1, 0)

        # cal scores
        recall    = recall_score(    valid_y, valid_pred)
        precision = precision_score( valid_y, valid_pred)
        auc_score = roc_auc_score(   valid_y, valid_prob)

        # append scores
        models.append(model)
        recalls.append(recall)
        precisions.append(precision)
        auc_scores.append(auc_score)
        
    return models, auc_scores, recalls, precisions

In [54]:
# loss 비교를 위해 지우지 않습니다.
# 최고점 모델입니다.
models, auc_scores, _, _ = catb_fold_train_pred(X_temp, train_b_p.target)
print(np.mean(auc_scores))

0.7311500000000001


In [57]:
pred_y_list = []
for model in models:
    pred_y = model.predict_proba(y_temp)
    pred_y_list.append(pred_y[:, 1].reshape(-1,1))
    
pred_ensemble_catb = np.mean(pred_y_list, axis = 0)

In [58]:
pred_ensemble_catb

array([[0.94895007],
       [0.22181754],
       [0.48093417],
       ...,
       [0.71755727],
       [0.89595603],
       [0.41866161]])

In [285]:
# submission.problem = pred_ensemble
# submission

,user_id,problem
0,30000,0.953445
1,30001,0.237317
2,30002,0.474520
3,30003,0.838170
4,30004,0.913806
...,...,...
14994,44994,0.208481
14995,44995,0.302758
14996,44996,0.727264
14997,44997,0.911754


In [59]:
# submission.to_csv(r'./submission/team_210130_2_pycaret-catb.csv', index=False)

### Tuned LGBM

In [60]:
#------------------------------------------------------------
# validation auc score를 확인하기 위해 정의
def f_pr_auc(probas_pred, y_true):
    labels=y_true.get_label()
    p, r, _ = precision_recall_curve(labels, probas_pred)
    score=auc(r,p) 
    return "pr_auc", score, True
#------------------------------------------------------------


def lgbm_fold_train_pred(train_x, train_y):

    # Train
    models     = []
    recalls    = []
    precisions = []
    auc_scores   = []
    threshold = 0.5
    # 파라미터 설정
    params =      {
                    'boosting_type':'gbdt', 
                    'class_weight':None,
                    'colsample_bytree':1.0,
                    'importance_type':'split',
                    'learning_rate':0.1,
                    'max_depth':-1,
                    'min_child_samples':20,
                    'min_child_weight':0.001,
                    'min_split_gain':0.0,
                    'n_estimators':100,
                    'n_jobs':-1,
                    'num_leaves':31,
                    'objective':None,
                    'random_state':2584,
                    'reg_alpha':0.0,
                    'reg_lambda':0.0,
                    'silent':True,
                    'subsample':1.0,
                    'subsample_for_bin':200000,
                    'subsample_freq':0
                    }
    #-------------------------------------------------------------------------------------
    # 5 Kfold cross validation
    s_fold = StratifiedKFold(n_splits=5, shuffle=True ,random_state=42)    

    for train_idx, val_idx in s_fold.split(train_x, train_y):

        # split train, validation set
        X = train_x.iloc[train_idx]
        y = train_y.iloc[train_idx]
        valid_x = train_x.iloc[val_idx]
        valid_y = train_y.iloc[val_idx]

        #run traning
        model = LGBMClassifier(**params)
        model.fit(X, y)

        # cal valid prediction
        valid_prob = model.predict(valid_x)
        valid_pred = np.where(valid_prob > threshold, 1, 0)

        # cal scores
        recall    = recall_score(    valid_y, valid_pred)
        precision = precision_score( valid_y, valid_pred)
        auc_score = roc_auc_score(   valid_y, valid_prob)

        # append scores
        models.append(model)
        recalls.append(recall)
        precisions.append(precision)
        auc_scores.append(auc_score)
        
    return models, auc_scores, recalls, precisions

In [61]:
# loss 비교를 위해 지우지 않습니다.
# 최고점 모델입니다.
models, auc_scores, _, _ = lgbm_fold_train_pred(X_temp, train_b_p.target)
print(np.mean(auc_scores))

0.7343500000000001


In [62]:
pred_y_list = []
for model in models:
    pred_y = model.predict_proba(y_temp)
    pred_y_list.append(pred_y[:, 1].reshape(-1,1))
    
pred_ensemble_lgbm = np.mean(pred_y_list, axis = 0)

In [63]:
pred_ensemble_lgbm

array([[0.9627417 ],
       [0.23273525],
       [0.51332842],
       ...,
       [0.7930191 ],
       [0.91029832],
       [0.38532945]])

In [64]:
# submission.problem = pred_ensemble
# submission

In [65]:
# submission.to_csv(r'./submission/team_210201_2_add-feature-made-by-hi.csv', index=False)

### Tuned GBC

In [66]:
gbc = GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=2584, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [74]:
## GBC
#-------------------------------------------------------------------------------------
# validation auc score를 확인하기 위해 정의
def f_pr_auc(probas_pred, y_true):
    labels=y_true.get_label()
    p, r, _ = precision_recall_curve(labels, probas_pred)
    score=auc(r,p) 
    return "pr_auc", score, True
#-------------------------------------------------------------------------------------
def gbc_fold_train_pred(train_x, train_y):

    # Train
    models     = []
    recalls    = []
    precisions = []
    auc_scores   = []
    threshold = 0.5

    k_fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    for train_idx, val_idx in k_fold.split(train_x, train_y):

        # split train, validation set
        X = train_x.iloc[train_idx]
        y = train_y.iloc[train_idx]
        valid_x = train_x.iloc[val_idx]
        valid_y = train_y.iloc[val_idx]

        #run traning
        model = gbc.fit(X, y)

        # cal valid prediction
        valid_prob = model.predict(valid_x)
        valid_pred = np.where(valid_prob > threshold, 1, 0)

        # cal scores
        recall    = recall_score(    valid_y, valid_pred)
        precision = precision_score( valid_y, valid_pred)
        auc_score = roc_auc_score(   valid_y, valid_prob)

        print(auc_score)

        # append scores
        models.append(model)
        recalls.append(recall)
        precisions.append(precision)
        auc_scores.append(auc_score)
    
    return models, auc_scores, recalls, precisions

In [75]:
# loss 비교를 위해 지우지 않습니다.
# 최고점 모델입니다.
models, auc_scores, _, _ = gbc_fold_train_pred(X_temp, train_b_p.target)
print(np.mean(auc_scores))

0.73125
0.73425
0.7155
0.72125
0.7200000000000001
0.72445


In [76]:
pred_y_list = []
for model in models:
    pred_y = model.predict_proba(y_temp)
    pred_y_list.append(pred_y[:, 1].reshape(-1,1))
    
pred_ensemble_gbc = np.mean(pred_y_list, axis = 0)

In [77]:
pred_ensemble_gbc

array([[0.93918089],
       [0.22982864],
       [0.51424097],
       ...,
       [0.68564711],
       [0.87796247],
       [0.36983057]])

### Tuned NGB

In [78]:
## ngboost
def f_pr_auc(probas_pred, y_true):
    labels = y_true.get_label()
    p, r, _ = precision_recall_curve(labels, probas_pred)
    score = auc(r, p)
    return "pr_auc", score, True

def ngb_fold_train_pred(train_x, train_y):

    models = []
    recalls = []
    precisions =[]
    auc_scores = []
    threshold = 0.5

    k_fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    for train_idx, val_idx in k_fold.split(train_x, train_y):
        X = train_x.iloc[train_idx]
        y = train_y.iloc[train_idx].astype(np.int)
        valid_x = train_x.iloc[val_idx]
        valid_y = train_y.iloc[val_idx].astype(np.int)

        model = NGBClassifier(
                                Dist = k_categorical(2),
                                n_estimators = 1000,
                                learning_rate = 0.02
                            )
        model.fit(X, y, X_val=valid_x, Y_val = valid_y, early_stopping_rounds = 3)
        valid_prob = model.predict_proba(valid_x)[:,1]
        valid_pred = np.where(valid_prob > threshold, 1, 0)

        recall = recall_score(valid_y, valid_pred)
        precision = precision_score(valid_y, valid_pred)
        auc_score = roc_auc_score(valid_y, valid_prob)

        models.append(model)
        recalls.append(recall)
        precisions.append(precision)
        auc_scores.append(auc_score)

    return models, auc_scores, recalls, precisions

In [79]:
# loss 비교를 위해 지우지 않습니다.
# 최고점 모델입니다.
models, auc_scores, _, _ = ngb_fold_train_pred(X_temp, train_b_p.target)
print(np.mean(auc_scores))

[iter 0] loss=0.6365 val_loss=0.6266 scale=2.0000 norm=4.0000
[iter 100] loss=0.4631 val_loss=0.4741 scale=2.0000 norm=3.6735
== Early stopping achieved.
== Best iteration / VAL170 (val_loss=0.4679)
[iter 0] loss=0.6365 val_loss=0.6263 scale=2.0000 norm=4.0000
[iter 100] loss=0.4649 val_loss=0.4612 scale=2.0000 norm=3.6630
[iter 200] loss=0.4534 val_loss=0.4543 scale=1.0000 norm=1.8491
== Early stopping achieved.
== Best iteration / VAL215 (val_loss=0.4540)
[iter 0] loss=0.6365 val_loss=0.6271 scale=2.0000 norm=4.0000
[iter 100] loss=0.4605 val_loss=0.4801 scale=2.0000 norm=3.6648
[iter 200] loss=0.4457 val_loss=0.4720 scale=2.0000 norm=3.7158
== Early stopping achieved.
== Best iteration / VAL211 (val_loss=0.4716)
[iter 0] loss=0.6365 val_loss=0.6267 scale=2.0000 norm=4.0000
[iter 100] loss=0.4633 val_loss=0.4725 scale=1.0000 norm=1.8378
[iter 200] loss=0.4470 val_loss=0.4623 scale=1.0000 norm=1.8602
== Early stopping achieved.
== Best iteration / VAL253 (val_loss=0.4603)
[iter 0] los

In [80]:
pred_y_list = []
for model in models:
    pred_y = model.predict_proba(y_temp)
    pred_y_list.append(pred_y[:, 1].reshape(-1,1))
    
pred_ensemble_ngb = np.mean(pred_y_list, axis = 0)

In [81]:
pred_ensemble_ngb

array([[0.97145546],
       [0.22713096],
       [0.45922968],
       ...,
       [0.62442086],
       [0.88451926],
       [0.27311474]])

### Tuned Xtree

In [83]:
#------------------------------------------------------------
# validation auc score를 확인하기 위해 정의
def f_pr_auc(probas_pred, y_true):
    labels=y_true.get_label()
    p, r, _ = precision_recall_curve(labels, probas_pred)
    score=auc(r,p) 
    return "pr_auc", score, True
#------------------------------------------------------------


def xtree_fold_train_pred(train_x, train_y):

    # Train
    models     = []
    recalls    = []
    precisions = []
    auc_scores   = []
    threshold = 0.5
    # 파라미터 설정
    model = ExtraTreesClassifier(
                                    bootstrap=False,
                                    ccp_alpha=0.0,
                                    class_weight=None,
                                    criterion='gini',
                                    max_depth=None,
                                    max_features='auto',
                                    max_leaf_nodes=None,
                                    max_samples=None,
                                    min_impurity_decrease=0.0,
                                    min_impurity_split=None,
                                    min_samples_leaf=1,
                                    min_samples_split=2,
                                    min_weight_fraction_leaf=0.0, 
                                    n_estimators=2000,
                                    n_jobs=-1,
                                    oob_score=False,
                                    random_state=2584,
                                    verbose=1,
                                    warm_start=False
                                )
    #-------------------------------------------------------------------------------------
    # 5 Kfold cross validation
    s_fold = StratifiedKFold(n_splits=5, shuffle=True ,random_state=42)    

    for train_idx, val_idx in s_fold.split(train_x, train_y):

        # split train, validation set
        X = train_x.iloc[train_idx]
        y = train_y.iloc[train_idx]
        valid_x = train_x.iloc[val_idx]
        valid_y = train_y.iloc[val_idx]

        #run traning
        model.fit(X, y)

        # cal valid prediction
        valid_prob = model.predict(valid_x)
        valid_pred = np.where(valid_prob > threshold, 1, 0)

        # cal scores
        recall    = recall_score(    valid_y, valid_pred)
        precision = precision_score( valid_y, valid_pred)
        auc_score = roc_auc_score(   valid_y, valid_prob)

        # append scores
        models.append(model)
        recalls.append(recall)
        precisions.append(precision)
        auc_scores.append(auc_score)

        print('==========================================================')
        
    return models, auc_scores, recalls, precisions

In [84]:
# loss 비교를 위해 지우지 않습니다.
# 최고점 모델입니다.
models, auc_scores, _, _ = xtree_fold_train_pred(X_temp, train_b_p.target)
print(np.mean(auc_scores))

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 152 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 402 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done 752 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 1202 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done 1752 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done 2000 out of 2000 | elapsed:    3.6s finished
[Parallel(n_jobs=24)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 152 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 402 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 752 tasks      | elapsed:    0.1s
[Parallel(n_jobs=24)]: Done 1202 tasks      | elapsed:    0.2s
[Parallel(n_jobs=24)]: Done 1752 tasks      | elapsed:    0.3s
[Parallel(n_job

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 152 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 402 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done 752 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 1202 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 1752 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done 2000 out of 2000 | elapsed:    3.6s finished
[Parallel(n_jobs=24)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 152 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 402 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 752 tasks      | elapsed:    0.1s
[Parallel(n_jobs=24)]: Done 1202 tasks      | elapsed:    0.2s
[Parallel(n_jobs=24)]: Done 1752 tasks      | elapsed:    0.3s
[Parallel(n_job

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 152 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 402 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done 752 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 1202 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 1752 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done 2000 out of 2000 | elapsed:    3.6s finished
[Parallel(n_jobs=24)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 152 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 402 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 752 tasks      | elapsed:    0.1s
[Parallel(n_jobs=24)]: Done 1202 tasks      | elapsed:    0.2s
[Parallel(n_jobs=24)]: Done 1752 tasks      | elapsed:    0.3s
[Parallel(n_job

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 152 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 402 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done 752 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 1202 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 1752 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done 2000 out of 2000 | elapsed:    3.7s finished
[Parallel(n_jobs=24)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 152 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 402 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 752 tasks      | elapsed:    0.1s
[Parallel(n_jobs=24)]: Done 1202 tasks      | elapsed:    0.2s
[Parallel(n_jobs=24)]: Done 1752 tasks      | elapsed:    0.3s
[Parallel(n_job

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 152 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 402 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done 752 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 1202 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done 1752 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done 2000 out of 2000 | elapsed:    3.7s finished
[Parallel(n_jobs=24)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 152 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 402 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 752 tasks      | elapsed:    0.1s
[Parallel(n_jobs=24)]: Done 1202 tasks      | elapsed:    0.2s


0.7247


[Parallel(n_jobs=24)]: Done 1752 tasks      | elapsed:    0.3s
[Parallel(n_jobs=24)]: Done 2000 out of 2000 | elapsed:    0.3s finished


In [86]:
pred_y_list = []
for model in models:
    pred_y = model.predict_proba(y_temp)
    pred_y_list.append(pred_y[:, 1].reshape(-1,1))
    
pred_ensemble_xtree = np.mean(pred_y_list, axis = 0)

[Parallel(n_jobs=24)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 152 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 402 tasks      | elapsed:    0.1s
[Parallel(n_jobs=24)]: Done 752 tasks      | elapsed:    0.2s
[Parallel(n_jobs=24)]: Done 1202 tasks      | elapsed:    0.4s
[Parallel(n_jobs=24)]: Done 1752 tasks      | elapsed:    0.6s
[Parallel(n_jobs=24)]: Done 2000 out of 2000 | elapsed:    0.6s finished
[Parallel(n_jobs=24)]: Using backend ThreadingBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 152 tasks      | elapsed:    0.0s
[Parallel(n_jobs=24)]: Done 402 tasks      | elapsed:    0.1s
[Parallel(n_jobs=24)]: Done 752 tasks      | elapsed:    0.2s
[Parallel(n_jobs=24)]: Done 1202 tasks      | elapsed:    0.3s
[Parallel(n_jobs=24)]: Done 1752 tasks      | elapsed:    0.5s
[Parallel(n_job

In [87]:
pred_ensemble_xtree

array([[0.906 ],
       [0.181 ],
       [0.5225],
       ...,
       [0.598 ],
       [0.6755],
       [0.4555]])

### Tuned RF

In [88]:
rfc = RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=7367, verbose=0,
                       warm_start=False)

In [90]:
def rf_fold_train_pred(train_x, train_y):
    
    models     = []
    recalls    = []
    precisions = []
    auc_scores   = []
    oob_scores   = []
    threshold = 0.5
    
    s_fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)  
    
    for train_idx, val_idx in s_fold.split(train_x, train_y):
        
        # split train, validation set
        X = train_x.iloc[train_idx]
        y = train_y.iloc[train_idx]
        valid_x = train_x.iloc[val_idx]
        valid_y = train_y.iloc[val_idx]

        #run traning
        model = rfc.fit(X, y)

        # cal valid prediction
        valid_prob = model.predict(valid_x)
        valid_pred = np.where(valid_prob > threshold, 1, 0)

        # cal scores
        recall    = recall_score(    valid_y, valid_pred)
        precision = precision_score( valid_y, valid_pred)
        auc_score = roc_auc_score(   valid_y, valid_prob)
        
        models.append(model)
        recalls.append(recall)
        precisions.append(precision)
        auc_scores.append(auc_score)
        
    return models, auc_scores, recalls, precisions

In [91]:
# loss 비교를 위해 지우지 않습니다.
# 최고점 모델입니다.
models, auc_scores, _, _ = rf_fold_train_pred(X_temp, train_b_p.target)
print(np.mean(auc_scores))

0.7246999999999999


In [92]:
pred_y_list = []
for model in models:
    pred_y = model.predict_proba(y_temp)
    pred_y_list.append(pred_y[:, 1].reshape(-1,1))
    
pred_ensemble_rf = np.mean(pred_y_list, axis = 0)

In [93]:
pred_ensemble_rf

array([[0.89],
       [0.2 ],
       [0.5 ],
       ...,
       [0.53],
       [0.72],
       [0.38]])

## Ensemble

In [94]:
pred_ensemble_catb

array([[0.94895007],
       [0.22181754],
       [0.48093417],
       ...,
       [0.71755727],
       [0.89595603],
       [0.41866161]])

In [95]:
pred_ensemble_gbc

array([[0.93918089],
       [0.22982864],
       [0.51424097],
       ...,
       [0.68564711],
       [0.87796247],
       [0.36983057]])

In [96]:
pred_ensemble_lgbm

array([[0.9627417 ],
       [0.23273525],
       [0.51332842],
       ...,
       [0.7930191 ],
       [0.91029832],
       [0.38532945]])

In [97]:
pred_ensemble_ngb

array([[0.97145546],
       [0.22713096],
       [0.45922968],
       ...,
       [0.62442086],
       [0.88451926],
       [0.27311474]])

In [98]:
pred_ensemble_rf

array([[0.89],
       [0.2 ],
       [0.5 ],
       ...,
       [0.53],
       [0.72],
       [0.38]])

In [99]:
pred_ensemble_xtree

array([[0.906 ],
       [0.181 ],
       [0.5225],
       ...,
       [0.598 ],
       [0.6755],
       [0.4555]])

In [100]:
pred_ensemble_best_6 = (
                            pred_ensemble_catb +
                            pred_ensemble_gbc  +
                            pred_ensemble_lgbm +
                            pred_ensemble_ngb  +
                            pred_ensemble_rf   +
                            pred_ensemble_xtree
                        ) / 6

In [101]:
pred_ensemble_best_6

array([[0.93638802],
       [0.21541873],
       [0.49837221],
       ...,
       [0.65810739],
       [0.82737268],
       [0.38040606]])

In [102]:
submission.problem = pred_ensemble_best_6
submission

,user_id,problem
0,30000,0.936388
1,30001,0.215419
2,30002,0.498372
3,30003,0.729066
4,30004,0.861503
...,...,...
14994,44994,0.223375
14995,44995,0.287214
14996,44996,0.658107
14997,44997,0.827373


In [103]:
submission.to_csv(r'./submission/team_210201_3_ensemble-best6.csv', index=False)

# 창고

In [42]:
# # 형록이가 제안한 var 모델

# X = pd.concat([train_e.user_id, pd.get_dummies(train_e.errtype)], axis=1).groupby('user_id').var().fillna(0)
# y = pd.concat([test_e.user_id, pd.get_dummies(test_e.errtype)], axis=1).groupby('user_id').var().fillna(0)

# X.columns = ['et_1', 'et_2', 'et_3', 'et_4', 'et_5', 'et_6', 'et_7', 'et_8', 'et_9', 'et_10', 'et_11', 'et_12', 'et_13',
#  'et_14', 'et_15', 'et_16', 'et_17', 'et_18', 'et_19', 'et_20', 'et_21', 'et_22', 'et_23', 'et_24', 'et_25',
#  'et_26', 'et_27', 'et_28', 'et_30', 'et_31', 'et_32', 'et_33', 'et_34', 'et_35', 'et_36', 'et_37', 'et_38',
#  'et_39', 'et_40', 'et_41', 'et_42']
# y.columns = ['et_1', 'et_2', 'et_3', 'et_4', 'et_5', 'et_6', 'et_7', 'et_8', 'et_9', 'et_10', 'et_11', 'et_12', 'et_13',
#  'et_14', 'et_15', 'et_16', 'et_17', 'et_18', 'et_19', 'et_20', 'et_21', 'et_22', 'et_23', 'et_24', 'et_25',
#  'et_26', 'et_27', 'et_28', 'et_30', 'et_31', 'et_32', 'et_33', 'et_34', 'et_35', 'et_36', 'et_37', 'et_38',
#  'et_39', 'et_40', 'et_41', 'et_42']

In [230]:
# 혜인이가 제안한 errcode

# # 8호
# train_ecode_14 = train_e[train_e.errtype == 14][['user_id', 'errcode', 'time']].groupby(['user_id', 'errcode']).count().unstack().fillna(0)
# test_ecode_14 = test_e[test_e.errtype == 14][['user_id', 'errcode', 'time']].groupby(['user_id', 'errcode']).count().unstack().fillna(0)
# train_ecode_14.columns = ['err_14_1', 'err_14_13','err_14_14']
# test_ecode_14.columns =  ['err_14_1', 'err_14_13','err_14_14']

# # 30호
# train_ecode_30 = train_e[train_e.errtype == 30][['user_id', 'errcode', 'time']].groupby(['user_id', 'errcode']).count().unstack().fillna(0)
# test_ecode_30 = test_e[test_e.errtype == 30][['user_id', 'errcode', 'time']].groupby(['user_id', 'errcode']).count().unstack().fillna(0)
# train_ecode_30.columns = ['err_30_0','err_30_1','err_30_2', 'err_30_3','err_30_4']
# test_ecode_30.columns =  ['err_30_0','err_30_1','err_30_2', 'err_30_3','err_30_4']

# # 9호
# train_ecode_9 = train_e[train_e.errtype == 9][['user_id', 'errcode', 'time']].groupby(['user_id', 'errcode']).count().unstack().fillna(0)
# test_ecode_9 = test_e[test_e.errtype == 9][['user_id', 'errcode', 'time']].groupby(['user_id', 'errcode']).count().unstack().fillna(0)
# train_ecode_9.columns = ['err_9_1','err_9_C-11020', 'err_9_C-12032','err_9_C-13039','err_9_C-14014',
#                           'err_9_V-21002','err_9_V-21004','err_9_V-21005','err_9_V-21008']
# test_ecode_9.columns = ['err_9_1','err_9_C-11020', 'err_9_C-12032','err_9_C-13039','err_9_C-14014',
#                           'err_9_V-21002','err_9_V-21004','err_9_V-21005','err_9_V-21008']

# # 37호
# train_ecode_37 = train_e[train_e.errtype == 37][['user_id', 'errcode', 'time']].groupby(['user_id', 'errcode']).count().unstack().fillna(0)
# test_ecode_37 = test_e[test_e.errtype == 37][['user_id', 'errcode', 'time']].groupby(['user_id', 'errcode']).count().unstack().fillna(0)
# train_ecode_37.columns = ['err_37_1', 'err_37_2']
# test_ecode_37.columns = ['err_37_1', 'err_37_2']


# # 5호
# train_ecode_5 = train_e[train_e.errtype == 5][['user_id', 'errcode', 'time']].\
#                             groupby(['user_id', 'errcode']).count().unstack().fillna(0)[('time', 'E-59902')]
# test_ecode_5 = test_e[test_e.errtype == 5][['user_id', 'errcode', 'time']].\
#                             groupby(['user_id', 'errcode']).count().unstack().fillna(0)[('time', 'E-59902')]
# #train_ecode_5.columns = ['err_5_E-59902']
# #test_ecode_5.columns = ['err_5_E-59902']
# train_ecode_5 = train_ecode_5.rename(level = 0, index = 'err_5_E-59902') 
# test_ecode_5 = test_ecode_5.rename(level = 0, index = 'err_5_E-59902') 

# # 31호
# train_ecode_31 = train_e[train_e.errtype == 31][['user_id', 'errcode', 'time']].\
#                             groupby(['user_id', 'errcode']).count().unstack().fillna(0)
# test_ecode_31 = test_e[test_e.errtype == 31][['user_id', 'errcode', 'time']].\
#                             groupby(['user_id', 'errcode']).count().unstack().fillna(0)
# train_ecode_31.columns = ['err_31_0', 'err_31_1']
# test_ecode_31.columns =['err_31_0', 'err_31_1']

# # 7호
# train_ecode_7 = train_e[train_e.errtype == 7][['user_id', 'errcode', 'time']].\
#                             groupby(['user_id', 'errcode']).count().unstack().fillna(0)
# test_ecode_7 = test_e[test_e.errtype == 7][['user_id', 'errcode', 'time']].\
#                             groupby(['user_id', 'errcode']).count().unstack().fillna(0)
# train_ecode_7.columns = ['err_7_1', 'err_7_14']
# test_ecode_7.columns =['err_7_1', 'err_7_14']

# # 25호
# train_ecode_25 = train_e[train_e.errtype == 25][['user_id', 'errcode', 'time']].groupby(['user_id', 'errcode']).count().unstack().fillna(0)
# test_ecode_25 = test_e[test_e.errtype == 25][['user_id', 'errcode', 'time']].groupby(['user_id', 'errcode']).count().unstack().fillna(0)

# train_ecode_25.columns =  ['err_25_1', 'err_25_2', 'err_25_L2CAP connection cancelled', 'err_25_UNKNOWN', 
#                             'err_25_connection fail to establish',
#                              'err_25_connection timeout', 'err_25_connectionterminated by local host',
#                             'err_25_scanning timeout', 'err_25_terminate by peer user']

# test_ecode_25.columns = ['err_25_1', 'err_25_2', 'err_25_L2CAP connection cancelled', 'err_25_UNKNOWN', 
#                            'connection fail for LMP response timout',
#                             'err_25_connection fail to establish',
#                              'err_25_connection timeout', 'err_25_connectionterminated by local host',
#                             'err_25_scanning timeout', 'err_25_terminate by peer user']

# test_ecode_25.drop('connection fail for LMP response timout', axis=1, inplace=True)